In [1]:
#| default_exp config

from nbdev.showdoc import show_doc
from utilities.ipynb_docgen import *

Implements:

- Cache
- Config
- MJD
- UTC, UTCnow

In [2]:
#| export
import os, sys, warnings, pickle
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple
import numpy as np

## Cache

In [3]:
#| export

class Cache(dict):
    """
    Manage a file cache

    - `path` -- string or `filepath` object <br> This is the folder where the index and data files are saved.
    - `clear` -- set True to clear the cache on initialization

    This uses pickle to save objects, associated with a hashable key, which is used to index the
    filename in a file `index.pkl` in the same folder.

    The `__call__` function is a convenient way to use it, so one call may either store a new entry or retrieve an existing one.

    """

    def __init__(self, path, clear:bool=False):

        self.path = Path(path) if path else None
        if self.path is None: return
        if not self.path.exists() :
            print(f'Warning: cache Path {self.path} does not exist, cache disabled ',file=sys.stderr)
            self.path=None
            return

        self.index_file = self.path/'index.pkl'

        if self.path.exists():
            if clear:
                print('Clearing cache!')
                self.clear()
            else:
                self._load_index()

    def _dump_index(self):
        with open(self.index_file, 'wb') as file:
            pickle.dump(self, file)

    def _load_index(self):
        if not self.index_file.exists():
            self._dump_index()
            return
        with open(self.index_file, 'rb') as file:
            self.update(pickle.load(file))

    def add(self, key, object,  exist_ok=False):
        if not self.path: return
        assert type(key)==str, f'Expect key to be a string, got {key}'
        if key  in self:
            if not exist_ok:
                print(f'Warning: cached object for key "{key}" exists', file=sys.stderr)
            filename = self[key]
        else:
            filename = f'cache_file_{hex(key.__hash__())[3:]}.pkl'
            self[key] = filename
            self._dump_index()

        with open(self.path/filename, 'wb') as file:
            pickle.dump(object, file )


    def get(self, key):
        if key not in self:
            return None
        filename = self[key]
        # temp fix
        if str(filename)[0]=='/': filename=self.path/filename.name

        if not (self.path/filename).exists():
            # perhaps deleted by another instance?
            print(f'File for Cache key {key} not found, removing entry', file='sys.stderr')
            selt.pop(key)
            return None
        with open(self.path/filename, 'rb') as file:
            ret = pickle.load(file)
        return ret

    def clear(self):
        if not self.path: return
        for f in self.path.iterdir():
            if f.is_file:
                f.unlink()
        super().clear()

        self._dump_index()

    def remove(self, key):
        """remove entry and associated file"""
        if not self.path: return
        if key not in self:
            print(f'Cache: key {key} not found', file=sys.stderr)
            return
        filename = self.path/self[key]
        try:
            filename.unlink()
        except:
            print(f'Failed to unlink file {filename}', file=sys.stderr)
            raise
        super().pop(key)
        self._dump_index()


    def __call__(self, key, func, *pars, description='', overwrite=False, **kwargs,
                ):
        """
        One-line usage interface for cache use

        - `key` -- key to use, usually a string. Must be hashable <br>
            If None, ignore cache and return the function evaluation
        - `func` -- user function that will return an object that can be pickled
        - `pars`, `kwargs` -- pass to `func`
        - `description` -- optional string that will be printed
        - `overwrite` -- if set, overwrite previous entry if exists

        Example:
        <pre>
        mycache = Cache('/tmp/thecache', clear=True)

        def myfun(x):
            return x

        result = mycache('mykey', myfun, x=99,  description='My data')

        </pre>

        """

        if key is None or self.path is None:
            return func(*pars, **kwargs)


        if description:
            print(f'{description}: {"Saving to" if key not in self or overwrite else "Restoring from"} cache', end='')
            print('' if key == description else f' with key "{key}"')
        ret = self.get(key)
        if ret is None or overwrite:
            ret = func(*pars, **kwargs)
            self.add(key, ret, exist_ok=overwrite)
        return ret

    def show(self, starts_with=''):
        import datetime
        if not self.path: return 'Cache not enabled'
        if len(self.items())==0: return f'Cache at {self.path} is empty\n'
        title = 'Cache contents' if not starts_with else f'Cache entries starting with {starts_with}'
        s = f'{title}\n {"key":30}   {"size":>10}  {"time":20} {"name"}, folder {self.path}\n'
        for name, value in self.items():
            # temporary: override file's path if set
            if str(value)[0]=='/':
                value = self.path/value.name
            if name is None or not name.startswith(starts_with) : continue
            try:
                stat = (self.path/value).stat()
                size = stat.st_size
                mtime= str(datetime.datetime.fromtimestamp(stat.st_mtime))[:16]
                s += f'  {name:30s}  {size:10}  {mtime:20} {value}\n'
            except Exception as msg:
                s += f'{name} -- file not found\n'
                raise
        return s

    def __str__(self):
        return self.show()

In [4]:
show_doc(Cache.__call__)

---

[source](https://github.com/tburnett/wtlike/tree/master/blob/master/wtlike/config.py#L115){target="_blank" style="float:right; font-size:smaller"}

### Cache.__call__

>      Cache.__call__ (key, func, *pars, description='', overwrite=False,
>                      **kwargs)

One-line usage interface for cache use

- `key` -- key to use, usually a string. Must be hashable <br>
    If None, ignore cache and return the function evaluation
- `func` -- user function that will return an object that can be pickled
- `pars`, `kwargs` -- pass to `func`
- `description` -- optional string that will be printed
- `overwrite` -- if set, overwrite previous entry if exists

Example:
<pre>
mycache = Cache('/tmp/thecache', clear=True)

def myfun(x):
    return x

result = mycache('mykey', myfun, x=99,  description='My data')

</pre>

### Cache test

In [5]:
#| code-fold: true
def cache_test(path):
    c = Cache(path, clear=True)

    # simmple interface
    c.add('one', 'one');
    c.add('two', 'two')
    c.add('two', 'two') # getnerates warning
    if path is not None:
        assert c.get('two') == 'two'

    # test function interface
    func = lambda x:f'value: {x}'
    
    r1 = c('four',  func,  4, description='Test')
    r2 = c('four',  func,  5,  description='Test') #should not get called
    assert c.path is None or r1==r2, f'{r1}, {r2}'
    
    # remaving an entry
    print(f'Before remove:\n{c}')
    assert 'four' in c
    c.remove('four')
    assert 'four' not in c
    print(f'After remove:\n{c}')
    c.clear()

test_path = Path('/tmp/cache_test')
test_path.mkdir(exist_ok=True)
cache_test(test_path)

Clearing cache!
Test: Saving to cache with key "four"
Test: Restoring from cache with key "four"
Before remove:
Cache contents
 key                                    size  time                 name, folder /tmp/cache_test
  one                                     18  2023-02-17 13:54     cache_file_736344f2a0a79ed8.pkl
  two                                     18  2023-02-17 13:54     cache_file_308134bc84799bf.pkl
  four                                    23  2023-02-17 13:54     cache_file_a6474c2549baaa0.pkl

After remove:
Cache contents
 key                                    size  time                 name, folder /tmp/cache_test
  one                                     18  2023-02-17 13:54     cache_file_736344f2a0a79ed8.pkl
  two                                     18  2023-02-17 13:54     cache_file_308134bc84799bf.pkl



In [6]:
#| export
class Config():
    defaults=\
    """
        verbose         : 1 # set to zero for no output
        warnings        : ignore #

        datapath        : ~/wtlike-data # where to find data--must be set
        cachepath       : ~/.cache/wtlike #

        # Expect 4FGL FITS file, e.g.,  gll_psc_v28.fit
        catalog_file    :

        # multiprocessing
        pool_size       : 1 # number of pool processes to use

        # data cuts, processing
        radius          : 4
        cos_theta_max   : 0.4
        z_max           : 100
        offset_size     : 2.e-06  # scale factor used for event time

        # binning -- actually determined by weight run
        energy_edge_pars : [2,6,17] # pars for np.logspace
        etypes          : [0, 1] # front, back
        nside           : 1024
        nest            : True

        # data selection for cell creation
        week_range      : []  # default all weeks found
        time_bins       : [0, 0, 7] # full MJD range, 7-day cells
        exp_min         : 5    # threshold for exposure per day, in cm^2 Ms units.

        # cell fitting
        use_kerr        : True  # Use the Kerr power-law exposure weighting
        likelihood_rep  : poisson
        poisson_tolerance : 0.2

    """

    def __init__(self, **kwargs):
        import yaml, warnings
        from yaml import SafeLoader

        # parameters: first defaults, then from ~/.config/wtlike/config.yaml, then kwars
        pars = yaml.load(self.defaults, Loader=SafeLoader)
        dp = Path('~/.config/wtlike/config.yaml').expanduser()
        if dp.is_file():
            with open(dp, 'r') as inp:
                userpars = yaml.load(inp, Loader=SafeLoader)
            pars.update(userpars)
            #print(f'update from user file {dp}: {userpars}')
        pars.update(kwargs)

        self.__dict__.update(pars)

        # set warnings filter if requested
        if self.warnings is not None and self.warnings != 'None':
            #print(f'*** set warnings filter to "{self.warnings}" ***')
            warnings.simplefilter(self.warnings)

        self.energy_edges = ee=np.logspace(*self.energy_edge_pars)
        self.energy_bins = np.sqrt(ee[1:] * ee[:-1])
        if not self.week_range:
            self.week_range = (None, None)

       # set up, check files paths
        self.error_msg=''
        if self.datapath is None:
            self.error_msg+='\ndatapath must be set to a folder with wtlike data'

        else:
            self.datapath = df = Path(self.datapath).expanduser()
            if not (self.datapath.is_dir() or  self.datapath.is_symlink()):
                self.error_msg+=f'\ndata_folder "{df}" is not a directory or symlink'
            subs = 'aeff_files weight_files data_files'.split()
            if self.error_msg=='':
                for sub in subs:
                    if not ( (df/sub).is_dir() or  (df/sub).is_symlink()) :
                        self.error_msg+=f'\n{df/sub} is not a directory or symlink'

        self.cachepath =  Path(self.cachepath).expanduser()
        os.makedirs(self.cachepath, exist_ok=True)
        if not self.cachepath.is_dir():
            self.error_msg +=f'cachepath {self.cachepath} is not a folder.'

        # look for 4FGL catalog file, gll_psc_v31.fit currently
        fail = False
        if self.catalog_file is None or self.datapath is None:
            t = Path(self.datapath).expanduser()
            u = sorted(list(t.glob('gll_psc_v*.fit')))
            if len(u)>0:
                self.catalog_file = u[-1]
            else:
                fail = True
        elif Path(self.catalog_file).expanduser().is_file():
            self.catalog_file = Path(self.catalog_file).expanduser()
        else: fail=True

        if fail:
            warnings.warn('There is no link to 4FGL catalog file: set "catalog_file" in your config.yaml'
                  ' or specify if in the Config() call')


    @property
    def cache(self):
        if not hasattr(self, '_cache'):
            self._cache = Cache(self.cachepath, clear=False)
        return self._cache

    @property
    def valid(self):
        if len(self.error_msg)==0: return True
        print(f'wtlike configuration is invalid:\n{self.error_msg}',file=sys.stderr)
        return False

    def __str__(self):
        s = 'Configuration parameters \n'
        for name, value in self.__dict__.items():
            if name=='files' or name.startswith('_'): continue
            s += f'  {name:15s} : {value}\n'
        return s

    def __repr__(self): return str(self)
    def get(self, *pars): return self.__dict__.get(*pars)

In [7]:
show_doc(Config)

---

[source](https://github.com/tburnett/wtlike/tree/master/blob/master/wtlike/config.py#L178){target="_blank" style="float:right; font-size:smaller"}

### Config

>      Config (**kwargs)

Initialize self.  See help(type(self)) for accurate signature.

In [8]:
#| code-fold: true
@ipynb_doc
def config_summary():
    """
    
    #### config.Config -- parameters are from three sources:
    - defaults
    - the file `~/.config/wtlike/config.yaml` if it exists
    - keyword args in Config constructor. For example, to suppress all printout:
        ```
    config = Config(verbose=0)
    ```
    
    ##### Config defaults
    This is yaml-format, corresponding to `config.yaml`.
    {config_defaults}
    
    
    #### Config contents as set up here
    {config_text}   


    
    ##### config.cache -- a file cache
    The class `Cache`, available from `config.cache` implements a file cache in the folder
    `{config.cachepath}`
    
    {cache_text}
    """
    config = Config()
    config_defaults = Config.defaults
    config_text = monospace(config, summary='config parameter list')

    try:
        cache_text = monospace(config.cache, f'current cache contents ({len(config.cache)} entries.)' )
    except Exception as msg:
        cache_text = f'(Failed: {msg})'


    return locals()
config_summary() 

#### config.Config -- parameters are from three sources:
- defaults
- the file `~/.config/wtlike/config.yaml` if it exists
- keyword args in Config constructor. For example, to suppress all printout:
    ```
config = Config(verbose=0)
```

##### Config defaults
This is yaml-format, corresponding to `config.yaml`.

        verbose         : 1 # set to zero for no output
        warnings        : ignore #

        datapath        : ~/wtlike-data # where to find data--must be set
        cachepath       : ~/.cache/wtlike #

        # Expect 4FGL FITS file, e.g.,  gll_psc_v28.fit
        catalog_file    :

        # multiprocessing
        pool_size       : 1 # number of pool processes to use

        # data cuts, processing
        radius          : 4
        cos_theta_max   : 0.4
        z_max           : 100
        offset_size     : 2.e-06  # scale factor used for event time

        # binning -- actually determined by weight run
        energy_edge_pars : [2,6,17] # pars for np.logspace
        etypes          : [0, 1] # front, back
        nside           : 1024
        nest            : True

        # data selection for cell creation
        week_range      : []  # default all weeks found
        time_bins       : [0, 0, 7] # full MJD range, 7-day cells
        exp_min         : 5    # threshold for exposure per day, in cm^2 Ms units.

        # cell fitting
        use_kerr        : True  # Use the Kerr power-law exposure weighting
        likelihood_rep  : poisson
        poisson_tolerance : 0.2

    


#### Config contents as set up here
<details  class="nbdoc-description" >  <summary> config parameter list </summary>  <div style="margin-left: 25px;"><pre>Configuration parameters <br>  verbose         : 1<br>  warnings        : None<br>  datapath        : /mnt/c/Users/thbur/onedrive/fermi/wtlike-data<br>  cachepath       : /home/burnett/.cache/wtlike<br>  catalog_file    : /mnt/c/Users/thbur/onedrive/fermi/wtlike-data/gll_psc_v28.fit<br>  pool_size       : 4<br>  radius          : 4<br>  cos_theta_max   : 0.4<br>  z_max           : 100<br>  offset_size     : 2e-06<br>  energy_edge_pars : [2, 6, 17]<br>  etypes          : [0, 1]<br>  nside           : 1024<br>  nest            : True<br>  week_range      : (None, None)<br>  time_bins       : [0, 0, 7]<br>  exp_min         : 5<br>  use_kerr        : True<br>  likelihood_rep  : poisson<br>  poisson_tolerance : 0.2<br>  keep_pixels     : True<br>  energy_edges    : [1.00000000e+02 1.77827941e+02 3.16227766e+02 5.62341325e+02<br> 1.00000000e+03 1.77827941e+03 3.16227766e+03 5.62341325e+03<br> 1.00000000e+04 1.77827941e+04 3.16227766e+04 5.62341325e+04<br> 1.00000000e+05 1.77827941e+05 3.16227766e+05 5.62341325e+05<br> 1.00000000e+06]<br>  energy_bins     : [1.33352143e+02 2.37137371e+02 4.21696503e+02 7.49894209e+02<br> 1.33352143e+03 2.37137371e+03 4.21696503e+03 7.49894209e+03<br> 1.33352143e+04 2.37137371e+04 4.21696503e+04 7.49894209e+04<br> 1.33352143e+05 2.37137371e+05 4.21696503e+05 7.49894209e+05]<br>  error_msg       : <br></pre></div> </details>   



##### config.cache -- a file cache
The class `Cache`, available from `config.cache` implements a file cache in the folder
`/home/burnett/.cache/wtlike`

<details  class="nbdoc-description" >  <summary> current cache contents (435 entries.) </summary>  <div style="margin-left: 25px;"><pre>Cache contents<br> key                                    size  time                 name, folder /home/burnett/.cache/wtlike<br>  PSR J0835-4510_data              156120367  2022-06-06 12:55     cache_file_ae6854be273acdc.pkl<br>  PSR J0633+1746_data              121830254  2022-09-28 03:37     cache_file_5707380cef0a5b.pkl<br>  PSR J1709-4429_data              120303131  2022-06-06 13:05     cache_file_7500f4770cd889.pkl<br>  PSR J2021+4026_data              130940729  2022-06-06 13:11     cache_file_a8dc38f04aab018.pkl<br>  PSR J1836+5925_data              125537569  2022-06-06 13:17     cache_file_a2b8a18b80eb164.pkl<br>  PSR J2021+3651_data              126539471  2022-06-06 13:23     cache_file_6661649ffbd07a9e.pkl<br>  PSR J0007+7303_data              134394303  2022-07-04 09:49     cache_file_cfa096edb809c60.pkl<br>  PSR J1809-2332_data              116347883  2022-06-06 13:40     cache_file_fb3fdcb2262565.pkl<br>  PSR J1826-1256_data              122240951  2022-06-06 13:45     cache_file_335ec043a4024484.pkl<br>  PSR J1907+0602_data              118468339  2022-06-06 13:51     cache_file_55b744805b5c9ff9.pkl<br>  PSR J1418-6058_data              132775053  2022-06-06 13:58     cache_file_29d5c1a8500b93e8.pkl<br>  PSR J1057-5226_data              101444077  2022-06-06 14:03     cache_file_384e41fd4ebdac81.pkl<br>  PSR J1813-1246_data              117386407  2022-06-06 14:08     cache_file_08c2148395b6f1f.pkl<br>  PSR J1028-5819_data              122583649  2022-06-06 14:15     cache_file_812ad9ec583a961.pkl<br>  PSR J2229+6114_data              133820383  2022-06-06 14:21     cache_file_e476bf55a06fa3e.pkl<br>  PSR J1048-5832_data              120018647  2022-06-06 14:27     cache_file_70b0714d172f3f2.pkl<br>  PSR J1732-3131_data              123636019  2022-06-06 14:33     cache_file_19edcba9cfff048.pkl<br>  PSR J1413-6205_data              132165683  2022-06-06 14:40     cache_file_6b3b60386724b1db.pkl<br>  PSR J1747-2958_data              126607289  2022-06-06 14:46     cache_file_2f67d22cb93688bb.pkl<br>  PSR J1952+3252_data              114920081  2022-06-06 14:52     cache_file_512aa5183afaad0.pkl<br>  P88Y3243_data                     88630555  2022-06-07 05:16     cache_file_66a9d9663efb97.pkl<br>  P88Y5020_data                    119817913  2022-06-07 10:33     cache_file_29e7a2e2944fb978.pkl<br>  504H-0317_data                   138670728  2022-09-19 14:27     cache_file_cb1897be30807d5.pkl<br>  P88Y6266_data                     99892151  2022-06-13 17:57     cache_file_35907c8f78cbed7c.pkl<br>  P88Y2172_data                    117721945  2022-06-17 11:49     cache_file_175f18c1a956e4d.pkl<br>  P88Y0096_data                    133548253  2022-07-04 09:54     cache_file_3c73e1e34f4c55.pkl<br>  P88Y0521_data                     95747295  2022-07-02 09:46     cache_file_7f6ab463d3ea18fd.pkl<br>  P88Y4060_data                     99826151  2022-07-02 10:05     cache_file_43042fb37426e122.pkl<br>  P88Y1134_data                     94193191  2022-07-02 09:51     cache_file_24a8a2282c30fdea.pkl<br>  P88Y6057_data                     89569443  2022-07-02 10:10     cache_file_4c75eeb98fde1be8.pkl<br>  P88Y1710_data                    125661037  2022-07-02 09:56     cache_file_342f3a94f9ee1f36.pkl<br>  P88Y1932_data                     87756585  2022-07-02 10:00     cache_file_5d597895910d1291.pkl<br>  P88Y0643_data                     97581865  2022-07-03 08:42     cache_file_4f9f229b3e50c0d6.pkl<br>  P88Y0514_data                     96424943  2022-07-03 13:01     cache_file_49fb71b2c22b562d.pkl<br>  PSR J0633+1746_weeks_9-11           419065  2022-07-04 10:03     cache_file_dd5ac11dbd01293.pkl<br>  P88Y3157_weeks_9-165              20704939  2022-07-04 10:06     cache_file_cd697fdc73ce09.pkl<br>  PSR J0534+2200_data              111891877  2022-07-13 01:08     cache_file_737398512a45c22.pkl<br>  P88Y3157_data                     88180753  2022-07-13 01:15     cache_file_5f7b91e889cf5dcf.pkl<br>  P88Y0615_data                     94004743  2022-07-14 13:03     cache_file_19435ee6bf1a46a2.pkl<br>  PSR J2021+4026-nk                138951485  2022-07-15 09:24     cache_file_6e0c2e81f58fab52.pkl<br>  P88Y6266_weeks_10-110             15927363  2022-07-17 09:18     cache_file_76ee0485535b73f5.pkl<br>  P88Y3278_data                    124207381  2022-08-03 04:57     cache_file_26e31ac702da1f1.pkl<br>  P88Y4434_data                    106816689  2022-08-04 11:05     cache_file_f3a11f1afe0c9b5.pkl<br>  P88Y3250_data                    123716459  2022-08-07 15:12     cache_file_9b7a5cd8d294f31.pkl<br>  998F-0345_data                   128867947  2022-08-07 15:26     cache_file_4616ec497d573415.pkl<br>  P88Y4380_data                    109267690  2022-09-01 14:49     cache_file_5966d71954eb9c7c.pkl<br>  PSR J1227-4853_data               96895988  2022-09-06 10:13     cache_file_e34b9131c462818.pkl<br>  P88Y2514_data                    128075332  2022-09-06 10:18     cache_file_f031ae9f99c06a3.pkl<br>  PSR J2032+4127_data              138146218  2022-09-13 15:35     cache_file_a2b064fb8fd7a7d.pkl<br>  P7R43226_data                    134541190  2022-09-14 16:19     cache_file_d9a65e0603cd76c.pkl<br>  P88Y5623_data                    135227295  2022-09-15 07:15     cache_file_566327cc32de4a39.pkl<br>  PSR J2022+3842_data              137492341  2022-09-15 07:36     cache_file_afe3751a3f799e9.pkl<br>  P88Y5645_data                    138309136  2022-09-15 07:45     cache_file_cf7d3244f64888.pkl<br>  PSR J2030+3641_data              130030915  2022-09-15 07:54     cache_file_23ec22ceeb0ec6b.pkl<br>  PSR J2030+4415_data              131784065  2022-09-15 08:02     cache_file_2132ab534ae9e72f.pkl<br>  PSR J2034+3632_data              125880832  2022-09-15 08:10     cache_file_2400b39b11d3d9a.pkl<br>  P88Y5798_data                    128862153  2022-09-15 08:18     cache_file_2923d4ecfbe7bb4.pkl<br>  P88Y5811_data                    126757869  2022-09-15 08:36     cache_file_218fbe26adc90fe7.pkl<br>  P88Y5767_data                    127134210  2022-09-15 13:18     cache_file_c9f9903c8ea7c34.pkl<br>  03bF-0140_data                   130255216  2022-09-15 13:29     cache_file_19711e7f8e695404.pkl<br>  P88Y5864_data                    123108475  2022-09-15 13:59     cache_file_5617f93107eb19fb.pkl<br>  P88Y5586_data                    126057344  2022-09-15 14:32     cache_file_2acc1afb12e76aa8.pkl<br>  P88Y5559_data                    123088819  2022-09-15 14:49     cache_file_5c8020ae6d338f97.pkl<br>  P88Y5456_data                    111743989  2022-09-15 15:02     cache_file_75a608259dfae190.pkl<br>  P88Y5615_data                    118935600  2022-09-15 15:13     cache_file_a9111c0ea46158d.pkl<br>  504F-1318_data                   106929207  2022-09-15 16:01     cache_file_691a32c883370c.pkl<br>  605F-0459_data                   130806382  2022-09-15 17:12     cache_file_5c924fa9859457e6.pkl<br>  P88Y5544_data                    118054111  2022-09-15 17:35     cache_file_110986d8042d075.pkl<br>  P88Y5889_data                    111251245  2022-09-15 18:00     cache_file_e84ccd71e8b5cfb.pkl<br>  S8008-2189_data                  125620614  2022-09-16 04:10     cache_file_2b6eed9631066d6d.pkl<br>  204N-0143_data                   124219403  2022-09-16 04:16     cache_file_52978b23acd715da.pkl<br>  P88Y5832_data                    107320213  2022-09-16 04:40     cache_file_6b575f14c32c7762.pkl<br>  P88Y5726_data                    124461680  2022-09-16 05:33     cache_file_612ab78e52ee64c2.pkl<br>  998H-0179_data                   131844461  2022-09-16 06:02     cache_file_5b026b706657857.pkl<br>  PSR J2028+3332_data              120994958  2022-09-22 03:27     cache_file_839f1ede96d2da8.pkl<br>  S8008-2676_data                  118823432  2022-09-16 06:26     cache_file_a8d4e2d2a5b9b06.pkl<br>  P88Y5696_data                    113511510  2022-09-16 06:39     cache_file_13161aaea72dcce8.pkl<br>  P88Y5817_data                    128113559  2022-09-16 07:13     cache_file_4e11a3a72ca0d526.pkl<br>  504F-0961_data                   133263592  2022-09-16 07:29     cache_file_3d9e6364e70c5c62.pkl<br>  P88Y5751_data                    124622820  2022-09-16 08:35     cache_file_5a04f09baad0a97b.pkl<br>  PSR J2006+3102_data              121616166  2022-09-16 09:38     cache_file_efe6e9a7b8200af.pkl<br>  P88Y5881_data                    129739484  2022-09-16 09:51     cache_file_1e3b602e830ef16.pkl<br>  504S-0652_data                   121447137  2022-09-17 17:43     cache_file_7d041dbb05e267bb.pkl<br>  P88Y5611_data                    132614370  2022-09-18 04:57     cache_file_8e5f11a8818dfa6.pkl<br>  PSR J1957+5033_data              115594220  2022-09-18 05:21     cache_file_99969139e9d63a7.pkl<br>  P88Y5514_data                    116396301  2022-09-20 09:36     cache_file_ac74fea0aa700de.pkl<br>  P88Y5659_data                    122852366  2022-09-22 03:33     cache_file_3dbfd00ac514c20d.pkl<br>  P88Y1902_data                    124918479  2022-09-22 08:08     cache_file_434edd1a06b4fcdd.pkl<br>  PSR J2339-0533_data               86557678  2022-09-22 10:21     cache_file_63222dc72d4d23a4.pkl<br>  P88Y0960_data                    104830054  2022-09-22 11:29     cache_file_3eb07bc1bfc4985.pkl<br>  geminga_nokerr                   121830254  2022-09-28 15:29     cache_file_bef443beaa72c4f.pkl<br>  P88Y3560_data                    139179558  2022-10-12 15:14     cache_file_05b596be16297a5.pkl<br>  P88Y4680_data                    119344799  2022-10-13 11:08     cache_file_1e8566d669ae61cb.pkl<br>  P88Y4703_data                    123456235  2022-10-13 12:55     cache_file_0ec52d81853420d.pkl<br>  P88Y5296_data                     97637750  2022-10-14 04:34     cache_file_aaa256810fbc162.pkl<br>  P88Y6183_data                     98868833  2022-10-14 08:10     cache_file_36c714f2ab629f7.pkl<br>  P88Y4892_data                     98450541  2022-10-15 10:02     cache_file_5acd5ccd4bc4b6f7.pkl<br>  P88Y0152_data                    138862059  2022-10-15 13:44     cache_file_cd30b4fbda483a.pkl<br>  P88Y0494_data                    137314534  2022-10-15 13:54     cache_file_3ee71867eac7a7a.pkl<br>  605N-0514_data                   111723339  2022-10-15 14:04     cache_file_39a4073a1687399.pkl<br>  P88Y2551_data                    128854579  2022-10-15 14:12     cache_file_6dee0636b97a5562.pkl<br>  504P-0741_data                   128241883  2022-10-15 14:20     cache_file_1d20e5b91d4a4af2.pkl<br>  P88Y2676_data                    127743399  2022-10-15 14:28     cache_file_36239402a770ec4c.pkl<br>  504F-0978_data                   124793298  2022-10-15 14:36     cache_file_9ea641ef44ac0ac.pkl<br>  P88Y3045_data                    114578562  2022-10-15 14:43     cache_file_8864e0491fe7f8f.pkl<br>  P88Y3311_data                    129294368  2022-10-15 14:59     cache_file_336f9094e570810d.pkl<br>  P88Y3325_data                    129517535  2022-10-15 15:09     cache_file_23d07b75c8e89514.pkl<br>  P88Y3346_data                    131191641  2022-10-15 15:19     cache_file_4142e805730c3da5.pkl<br>  P88Y3380_data                    105826847  2022-10-15 15:28     cache_file_24c329f845cdfd4.pkl<br>  P88Y3505_data                    137628617  2022-10-15 15:37     cache_file_0ed7e8d4896f203.pkl<br>  P88Y3673_data                    139411972  2022-10-15 15:45     cache_file_4514acf144b466f5.pkl<br>  P88Y3738_data                    136645614  2022-10-15 15:54     cache_file_a038ba205dc2ee8.pkl<br>  03bP-0126_data                   132255249  2022-10-15 16:02     cache_file_786d9b5398680b4.pkl<br>  P88Y3956_data                    121982791  2022-10-15 20:27     cache_file_5fbf285f1b625fbc.pkl<br>  P88Y3971_data                     97078625  2022-10-15 20:33     cache_file_8d5bc3f85a4d6c.pkl<br>  P88Y4012_data                    110596773  2022-10-15 20:40     cache_file_b88f4e46118c0c9.pkl<br>  P88Y4015_data                    100259355  2022-10-15 20:47     cache_file_4154589aa3c4e75a.pkl<br>  504P-0675_data                   136520818  2022-10-16 04:36     cache_file_494a26215b14a58.pkl<br>  P88Y4138_data                    138606678  2022-10-16 04:41     cache_file_43e0a75ca2937a79.pkl<br>  P88Y4153_data                    129274173  2022-10-16 04:46     cache_file_d349fb0264b1f39.pkl<br>  P88Y4151_data                    137672157  2022-10-16 04:51     cache_file_373aff2d9919742.pkl<br>  P88Y4178_data                    139091673  2022-10-16 04:57     cache_file_b0b0b22af5a4787.pkl<br>  P88Y4191_data                    138507250  2022-10-16 05:03     cache_file_1bbc69df78d2947.pkl<br>  P88Y4305_data                    131551399  2022-10-16 05:09     cache_file_470c2cf440f46ca1.pkl<br>  P88Y4359_data                    136417183  2022-10-16 05:15     cache_file_a1fff908a72116.pkl<br>  P88Y0634_data                    125650805  2022-10-16 05:42     cache_file_4642daf3aabc4518.pkl<br>  211F-0560_data                   136581543  2022-10-17 13:17     cache_file_6fc8e3970e87e1c0.pkl<br>  P88Y4372_data                    133887313  2022-10-17 14:09     cache_file_8213a014b09a230.pkl<br>  P88Y4445_data                    127790530  2022-10-17 14:16     cache_file_c2dbe4cbdbb17c1.pkl<br>  P88Y4483_data                    105863926  2022-10-17 14:23     cache_file_52403d62668d410.pkl<br>  P88Y4505_data                    126788158  2022-10-17 14:30     cache_file_613aac9f248b1fd1.pkl<br>  P88Y4504_data                    118064240  2022-10-17 14:36     cache_file_3d1e2bffd74bea8b.pkl<br>  P88Y4550_data                    103571580  2022-10-17 14:43     cache_file_24fdddfc6e0db568.pkl<br>  P88Y4552_data                    122843840  2022-10-17 14:49     cache_file_7a6e96d2aefb5d5.pkl<br>  215P-0086_data                   128035362  2022-10-17 14:55     cache_file_40dbf580be6f5eca.pkl<br>  998P-0285_data                   128543247  2022-10-17 15:02     cache_file_213a79a2869a1cbc.pkl<br>  P88Y4692_data                    126636447  2022-10-17 15:09     cache_file_70b95e5ff41b190e.pkl<br>  P88Y4710_data                    116642799  2022-10-17 15:15     cache_file_8dd1ff0d6014227.pkl<br>  P88Y4714_data                     99976422  2022-10-17 15:20     cache_file_a8c9c731f63b8a6.pkl<br>  P88Y4728_data                    103430992  2022-10-17 15:25     cache_file_365035108f3a295.pkl<br>  P88Y4768_data                    133895356  2022-10-17 15:31     cache_file_67de064160a8efa2.pkl<br>  P88Y4822_data                    118132728  2022-10-17 15:36     cache_file_b5646314eca237.pkl<br>  011H-0115_data                   130455304  2022-10-17 15:42     cache_file_5597d427ad3758.pkl<br>  P88Y4867_data                    106287755  2022-10-17 15:48     cache_file_7c47237d2b04283.pkl<br>  P88Y4890_data                    101753246  2022-10-17 15:53     cache_file_0c18314c9bbf629.pkl<br>  S8008-2727_data                  113080317  2022-10-17 16:01     cache_file_7ceaa3a787672cc0.pkl<br>  P88Y4945_data                     98732193  2022-10-17 16:07     cache_file_7ae0c31a701197e.pkl<br>  P88Y4952_data                    127307551  2022-10-17 16:13     cache_file_37f08de9901a61a.pkl<br>  P88Y4967_data                     99223621  2022-10-17 16:19     cache_file_4a4cef27ec9fd90.pkl<br>  P88Y4978_data                    102303411  2022-10-17 16:25     cache_file_422f43018f979193.pkl<br>  P88Y5081_data                    127843905  2022-10-17 16:32     cache_file_cf7710d210c3acf.pkl<br>  P88Y5158_data                    126725130  2022-10-18 01:39     cache_file_64ce20a752de8def.pkl<br>  P88Y5192_data                    118642398  2022-10-18 01:44     cache_file_3faad2ec3f63048.pkl<br>  P88Y5200_data                    128270821  2022-10-18 01:50     cache_file_203c630ff791cdfd.pkl<br>  P88Y5213_data                    125955424  2022-10-18 01:55     cache_file_7e21a28571998b0.pkl<br>  P88Y5246_data                    129201709  2022-10-18 02:01     cache_file_76466490f9121c8.pkl<br>  03bP-0129_data                   128206526  2022-10-18 02:07     cache_file_cf4a07adf7b4e55.pkl<br>  P88Y5249_data                    126013209  2022-10-18 02:12     cache_file_51641a8689e820e.pkl<br>  P88Y5256_data                    127270458  2022-10-18 02:18     cache_file_deaac68ded1c4f.pkl<br>  P88Y5286_data                    126636741  2022-10-18 02:24     cache_file_769d6bc06f2c96d.pkl<br>  P88Y5315_data                    126039676  2022-10-18 02:30     cache_file_4191989690abdd16.pkl<br>  P88Y5313_data                    125568800  2022-10-18 02:36     cache_file_33fd0a4b66a5feb0.pkl<br>  P88Y5385_data                    126718207  2022-10-18 02:42     cache_file_15ab4cfc3051999a.pkl<br>  P88Y5403_data                    126439880  2022-10-18 02:48     cache_file_31d995274bf70a.pkl<br>  P88Y5502_data                    117268179  2022-10-18 02:54     cache_file_cbf16f9ff6acaf5.pkl<br>  P88Y5668_data                    113620332  2022-10-18 03:00     cache_file_6301172597348bb.pkl<br>  504S-0719_data                   129491866  2022-10-18 03:06     cache_file_3e59e1550b6bc608.pkl<br>  P88Y5926_data                    107658530  2022-10-18 03:12     cache_file_27372985a8bc4d47.pkl<br>  P88Y6145_data                    140295645  2022-10-18 03:19     cache_file_1d0d0f088b091278.pkl<br>  03bF-0106_data                   139699840  2022-10-18 03:31     cache_file_12edc69c385a51b5.pkl<br>  P88Y1769_data                    100740605  2022-10-18 03:36     cache_file_293191c7ed30.pkl<br>  P88Y1853_data                    101476550  2022-10-18 03:43     cache_file_93ea0c529cf880d.pkl<br>  011P-0162_data                   156932804  2022-10-18 03:49     cache_file_c32ee589181dee1.pkl<br>  P88Y2110_data                    136733177  2022-10-18 03:54     cache_file_6851a3b8dbb7a96a.pkl<br>  211N-0546_data                   126417501  2022-10-18 04:00     cache_file_430d6be40a805a30.pkl<br>  P8010-0487_data                  120102493  2022-10-18 04:06     cache_file_b6d67db9e88e76f.pkl<br>  P88Y2161_data                    123395349  2022-10-18 04:12     cache_file_5fa3e18aac17063b.pkl<br>  504P-0643_data                   122955070  2022-10-18 04:17     cache_file_13735e5a6666694.pkl<br>  211H-0166_data                   123431714  2022-10-18 04:23     cache_file_4cb3dc48fa107e0.pkl<br>  P88Y2613_data                    128307039  2022-10-18 04:29     cache_file_0ccd67754f84def.pkl<br>  03bF-0117_data                   119026684  2022-10-18 04:35     cache_file_d9ba881053db67.pkl<br>  P88Y3206_data                    127190917  2022-10-18 04:41     cache_file_6ec60ae80c6e2afd.pkl<br>  P88Y3326_data                    129344999  2022-10-18 04:48     cache_file_f2fce8f5c7ec9c0.pkl<br>  S8008-1708_data                  101459365  2022-10-18 04:54     cache_file_acd44f98b329c0.pkl<br>  S8010-0991_data                  138436648  2022-10-18 05:02     cache_file_5d885ae41cd123.pkl<br>  504P-0647_data                   138068903  2022-10-18 05:09     cache_file_5a6472c7d7679c7f.pkl<br>  211H-0177_data                   135914107  2022-10-18 05:17     cache_file_58a608aa43d2a0f.pkl<br>  P88Y4378_data                    133178416  2022-10-18 05:24     cache_file_c834fead52e1a96.pkl<br>  P88Y4754_data                    116019358  2022-10-18 05:31     cache_file_15241bcc4ff237e.pkl<br>  P88Y4806_data                    127808492  2022-10-18 05:38     cache_file_39bc815d1df6518.pkl<br>  P88Y4976_data                    129744118  2022-10-18 05:46     cache_file_4887ef08d7a99c3d.pkl<br>  PSR J1832-0836_data              128749572  2022-10-18 05:53     cache_file_7c3b9e70d0d3ce9c.pkl<br>  P88Y5093_data                    101349640  2022-10-18 06:02     cache_file_b1072108dbcf1c2.pkl<br>  215P-0060_data                   127809871  2022-10-18 06:22     cache_file_6df83f6fb98ea99.pkl<br>  P88Y5394_data                    127036525  2022-10-18 06:32     cache_file_6b7f1cdd15cf.pkl<br>  P88Y4776_data                    113872451  2022-10-22 12:46     cache_file_5b4e4b3c1c48b50a.pkl<br>  504N-1700_data                   127890049  2022-10-22 12:51     cache_file_70fcd1ea810c731f.pkl<br>  P88Y4766_data                    125640704  2022-10-22 12:57     cache_file_c49d8c8a4d15155.pkl<br>  605P-0459_data                   133224560  2022-10-22 13:03     cache_file_5659ff6ceb10ffc.pkl<br>  PSR J1824-2452A_data             106970598  2022-10-22 13:08     cache_file_2825d10243ea361.pkl<br>  P88Y2536_data                     88423888  2022-10-22 13:14     cache_file_6e819fcbe15857b.pkl<br>  P88Y2239_data                    111467125  2022-10-22 13:20     cache_file_3e3675afd09309.pkl<br>  P88Y4831_data                    131068896  2022-10-22 13:36     cache_file_82b7306b53b879a.pkl<br>  998N-0360_data                   112952560  2022-10-23 09:41     cache_file_90f340583e0c3d0.pkl<br>  P88Y0159_data                    128341010  2022-10-24 08:56     cache_file_2d559abc9cd6df9.pkl<br>  P88Y2298_data                     94524598  2022-10-24 09:01     cache_file_f27712e1c55e3c7.pkl<br>  P88Y4435_data                    108698807  2022-10-24 09:07     cache_file_384ef7d4615a2685.pkl<br>  211P-0526_data                   134840902  2022-10-24 09:25     cache_file_87329b9406f5145.pkl<br>  211P-0480_data                   133559531  2022-10-24 09:30     cache_file_15a816b1029fbdf1.pkl<br>  504F-1019_data                   130320134  2022-10-24 09:36     cache_file_0973524095109b.pkl<br>  998P-0253_data                   129901359  2022-10-24 09:43     cache_file_063690b18e2b8bd.pkl<br>  605N-0713_data                   110591565  2022-10-24 09:56     cache_file_5df457fd78d471.pkl<br>  605P-0675_data                   100539593  2022-10-24 10:02     cache_file_833fa5a154e5309.pkl<br>  04aP-0040_data                   124527823  2022-10-24 10:08     cache_file_2295c34f1147e5a7.pkl<br>  P88Y5232_data                     96311355  2022-10-24 10:14     cache_file_64219669713ad042.pkl<br>  605F-0715_data                   107105516  2022-10-24 10:19     cache_file_52798debf844b74.pkl<br>  P88Y5518_data                     96556621  2022-10-24 10:25     cache_file_309bcc69351cf980.pkl<br>  504N-0664_data                   117020806  2022-10-25 00:27     cache_file_96f8ce786e8e85a.pkl<br>  504F-1048_data                   121239139  2022-10-25 00:32     cache_file_29ed8da0d8207ae.pkl<br>  P88Y5280_data                    116674159  2022-10-25 00:43     cache_file_e6548623256a615.pkl<br>  PSR J1048+2339_data               97795929  2022-10-30 05:03     cache_file_563484ae6fd6cc07.pkl<br>  211P-0460_data                   124648496  2022-11-01 15:53     cache_file_6a8ce494e4e25b6.pkl<br>  211N-1338_data                   134704528  2022-11-01 16:00     cache_file_1d6d603567f55f08.pkl<br>  P88Y2089_data                    143325056  2022-11-01 16:07     cache_file_3e709d46cffe244.pkl<br>  211N-1337_data                   121527518  2022-11-01 16:15     cache_file_7823f38d70a6733f.pkl<br>  504N-0698_data                   130223863  2022-11-01 16:22     cache_file_b57092330e567fc.pkl<br>  504F-0957_data                   128967699  2022-11-01 16:30     cache_file_daeb215026ca40.pkl<br>  504H-0367_data                   126091805  2022-11-01 16:38     cache_file_3a94fc5b5f257a4c.pkl<br>  605P-0399_data                   121933973  2022-11-01 16:46     cache_file_979645508bcf8c.pkl<br>  504P-1395_data                   114890503  2022-11-01 16:54     cache_file_70ddc1bc79c0a309.pkl<br>  504F-1029_data                   124898494  2022-11-01 17:01     cache_file_dc411a7065eca7a.pkl<br>  504H-0264_data                   116142474  2022-11-01 17:07     cache_file_f1e3c2e91ed9248.pkl<br>  211N-0557_data                   130507013  2022-11-01 17:15     cache_file_d992543e95c9943.pkl<br>  211N-0572_data                   132126078  2022-11-01 17:22     cache_file_2bcfeffe37361ed5.pkl<br>  P88Y3453_data                    124473524  2022-11-01 17:28     cache_file_09f6e69697acd14.pkl<br>  04aP-0029_data                   137406948  2022-11-01 17:34     cache_file_61aa7d8314d7852.pkl<br>  504P-0546_data                   121455222  2022-11-01 17:40     cache_file_dd93cb00241a9dc.pkl<br>  204P-0147_data                   118572349  2022-11-01 17:46     cache_file_eae993552b327cf.pkl<br>  214F-0376_data                   116689328  2022-11-01 17:53     cache_file_7ef1c00db59b97d4.pkl<br>  504N-0640_data                   122360098  2022-11-01 17:58     cache_file_20ee925c4c5074ef.pkl<br>  211F-0404_data                   115293339  2022-11-01 18:04     cache_file_5522f1abed7c3f35.pkl<br>  S8008-2111_data                  127487696  2022-11-01 18:10     cache_file_078ebdf10fdef81.pkl<br>  S8008-1995_data                  120107512  2022-11-01 18:16     cache_file_38d2bd18b490ddf.pkl<br>  P88Y4200_data                    117113612  2022-11-01 18:22     cache_file_0c6f525b527382.pkl<br>  P88Y4205_data                    124090827  2022-11-01 18:27     cache_file_24ae000f371d3b5f.pkl<br>  P88Y4225_data                    112487599  2022-11-01 18:33     cache_file_6a5a8b21e35ad5c.pkl<br>  998F-0288_data                   113873410  2022-11-01 18:39     cache_file_6305c61bb1c2be8.pkl<br>  04aN-0059_data                   110860855  2022-11-01 18:45     cache_file_24069825a7abc24b.pkl<br>  03bN-0204_data                   134423856  2022-11-01 18:50     cache_file_37e5fa0f7aa261c7.pkl<br>  211F-0683_data                   118368600  2022-11-01 18:56     cache_file_31ff3414ffb1cb27.pkl<br>  P88Y4322_data                    125312432  2022-11-01 19:02     cache_file_47db06cffc543406.pkl<br>  605N-0746_data                   113421294  2022-11-01 19:08     cache_file_600da9f8a8e7004.pkl<br>  S8008-2365_data                  135606674  2022-11-01 19:14     cache_file_373864c5985d7176.pkl<br>  211P-0720_data                   107983806  2022-11-01 19:20     cache_file_20ac9be844da480.pkl<br>  605N-0556_data                   131480972  2022-11-01 19:26     cache_file_d8267c2da72d6cd.pkl<br>  P88Y4424_data                    117283054  2022-11-01 19:31     cache_file_3cf85cf4826bb4c1.pkl<br>  504P-0832_data                   121904139  2022-11-01 19:37     cache_file_5265a59fb6d9cf5.pkl<br>  211N-0625_data                   127653750  2022-11-01 19:43     cache_file_b42f4dc63b6cdba.pkl<br>  P88Y4662_data                    132675144  2022-11-01 19:49     cache_file_38cd671bcb82581b.pkl<br>  S8008-1918_data                  113436470  2022-11-01 19:54     cache_file_cd14ce4bd04d2ec.pkl<br>  504F-1114_data                   118379282  2022-11-01 20:00     cache_file_2539d1cb5291b9.pkl<br>  605S-0286_data                   101708215  2022-11-01 20:06     cache_file_8e1902a767d199.pkl<br>  204P-0066_data                   125591914  2022-11-01 20:12     cache_file_7edd2d1e938fa1dd.pkl<br>  504F-0777_data                   118091323  2022-11-01 20:17     cache_file_619db44ef0be30d8.pkl<br>  504P-0547_data                   120141686  2022-11-01 20:23     cache_file_d887ae8f6e92631.pkl<br>  998F-0265_data                   104259505  2022-11-01 20:29     cache_file_748d9a8d96f21e1.pkl<br>  605P-0862_data                   118016899  2022-11-01 20:34     cache_file_62a9b2942a7f87.pkl<br>  504P-0569_data                   119193473  2022-11-02 04:55     cache_file_cba9647eeb31c01.pkl<br>  211H-0227_data                   110925500  2022-11-02 04:59     cache_file_5f77bb2f4d7f7d6f.pkl<br>  504F-0765_data                   118164592  2022-11-02 05:04     cache_file_45fd54fd440ec35.pkl<br>  P88Y4877_data                    112681611  2022-11-02 05:09     cache_file_fcc6706a58f7fa4.pkl<br>  P88Y4904_data                    122809428  2022-11-02 05:14     cache_file_372f900a158cebf.pkl<br>  504N-0878_data                   121611014  2022-11-02 05:19     cache_file_a4de22d62278361.pkl<br>  S8010-0771_data                  120039080  2022-11-02 05:23     cache_file_4838f86c52635520.pkl<br>  211F-0815_data                   100448383  2022-11-02 05:28     cache_file_2b753ac1a1bf9aeb.pkl<br>  504F-0741_data                   117356638  2022-11-02 05:33     cache_file_7c498358fac07a61.pkl<br>  PSR J1857+0143_data              127769012  2022-11-02 05:38     cache_file_379f0cde8f39b835.pkl<br>  214P-0112_data                   122449915  2022-11-02 05:43     cache_file_d73f709364d4aec.pkl<br>  P88Y5342_data                    127011409  2022-11-02 05:48     cache_file_62af8e1477a065a.pkl<br>  P88Y5378_data                    127096263  2022-11-02 05:53     cache_file_4ea0a0ba0f1e806.pkl<br>  504F-0956_data                   126494130  2022-11-02 05:59     cache_file_07e6e638b68a395.pkl<br>  PSR J0437-4715_data               98828394  2022-11-30 08:29     cache_file_ad6b9dfdf1c769.pkl<br>  211N-1247_data                    87986969  2022-12-08 07:51     cache_file_4ef0b2370a8f222.pkl<br>  03bN-0309_data                   100197986  2022-12-08 08:44     cache_file_8631b25d81f331d.pkl<br>  211N-0034_data                   115954440  2022-12-09 11:48     cache_file_bf7462af32e11d6.pkl<br>  P88Y6099_data                    109066825  2022-12-09 12:12     cache_file_c9cbe45e21f42ab.pkl<br>  011F-0052_data                   102005869  2022-12-13 05:19     cache_file_5fd2bdea61df53e6.pkl<br>  S8008-2585_data                  135006347  2022-12-15 07:30     cache_file_d96ae5435ec0d8.pkl<br>  03bF-0101_data                   103286211  2022-12-15 07:34     cache_file_2e736d3a97c8d08.pkl<br>  211N-0694_data                   126941598  2022-12-15 07:39     cache_file_3496afb763ce2cb4.pkl<br>  211F-0356_data                   106728797  2022-12-15 07:43     cache_file_6963181d57a4da6a.pkl<br>  214P-0181_data                   138552946  2022-12-15 13:16     cache_file_3ae683c7726eacd.pkl<br>  211H-0196_data                   132771282  2022-12-15 13:40     cache_file_7f7efbf60cc23271.pkl<br>  211P-0513_data                   136528077  2022-12-15 13:44     cache_file_5cf0ba08e2b7600.pkl<br>  PSR J1702-4128_data              130886287  2022-12-15 13:51     cache_file_455dc15ef481f2e1.pkl<br>  215N-0077_data                   137598671  2022-12-15 15:04     cache_file_aacd99426c7c53d.pkl<br>  605F-1067_data                   137099284  2022-12-15 15:08     cache_file_d18b175226279d9.pkl<br>  03bH-0107_data                   136105270  2022-12-15 15:13     cache_file_6202ffa2b11340a.pkl<br>  P88Y4896_data                    128586108  2022-12-15 18:19     cache_file_e853c2825b6342.pkl<br>  211H-0192_data                   136510955  2022-12-19 06:38     cache_file_70142cf079ee231e.pkl<br>  204N-0140_data                   136982517  2022-12-19 07:18     cache_file_ef4715070417bf8.pkl<br>  P88Y4853_data                     97706777  2022-12-22 02:29     cache_file_737d062ca5312ef.pkl<br>  P88Y2746_data                    108772272  2022-12-27 05:36     cache_file_7410e5089644472.pkl<br>  P88Y2756_data                     99413629  2022-12-27 05:43     cache_file_5d12cc23c1fc5.pkl<br>  P88Y0652_data                    140761215  2022-12-27 14:08     cache_file_7c7e2788baec086.pkl<br>  605N-0275_data                   100739016  2023-01-04 04:55     cache_file_122f133d48ba6793.pkl<br>  P88Y5511_data                    122896802  2023-01-09 03:38     cache_file_9b0360f2f5ba800.pkl<br>  P88Y5823_data                    108920238  2023-01-10 08:04     cache_file_423781720caf9052.pkl<br>  P88Y5806_data                    110803749  2023-01-10 08:20     cache_file_5acc8daea8e184a.pkl<br>  214H-0056_data                   126879410  2023-01-10 09:47     cache_file_533669007340224b.pkl<br>  998S-0264_data                   131590564  2023-01-11 10:22     cache_file_7f63f8f8660c9c73.pkl<br>  PSR J1748-2815_data              135888228  2023-01-11 21:25     cache_file_21ba9ec71a0369ae.pkl<br>  PSR J1016-5857_data              129917074  2023-01-13 06:52     cache_file_f8bbfe2f3da45b0.pkl<br>  605H-0201_data                   112704669  2023-01-13 12:47     cache_file_8a06097396aa9cd.pkl<br>  211P-0299_data                   114944998  2023-01-13 15:03     cache_file_7ce16c8d0611eb4.pkl<br>  PSR J1410-6132_data              143533950  2023-01-14 09:54     cache_file_6bb213ba583df3c.pkl<br>  PSR J1420-6048_data              142736783  2023-01-14 10:04     cache_file_92536bd99985f39.pkl<br>  011P-0104_data                   123575116  2023-01-16 12:25     cache_file_c70b68bf32e6fd.pkl<br>  PSR J1913+1011_data              127877085  2023-01-16 14:32     cache_file_8a9332cd66d205c.pkl<br>  P88Y5133_data                    127095990  2023-01-18 00:21     cache_file_61ae4fc00a9754ff.pkl<br>  211N-0373_data                   114845752  2023-01-20 00:07     cache_file_81fd164cbc67523.pkl<br>  211F-0724_data                   113865346  2023-01-20 05:25     cache_file_75265e1604e0570c.pkl<br>  404P-0586_data                   100201122  2023-01-20 05:31     cache_file_4669e540caf03b2.pkl<br>  PSR J1023+0038_data               89818701  2023-01-20 05:39     cache_file_1551bc9826a5c32.pkl<br>  404N-0904_data                    98547204  2023-01-20 05:57     cache_file_4ba7c8769067518.pkl<br>  P88Y3381_data                     95705960  2023-01-24 17:33     cache_file_f9da1f96fcdcc2b.pkl<br>  504S-0941_data                   113847566  2023-01-28 05:32     cache_file_1441fc4af24d385.pkl<br>  504P-1136_data                   100150526  2023-01-28 05:35     cache_file_7bd4aca4db5990b5.pkl<br>  404S-0434_data                   101165911  2023-01-28 05:42     cache_file_3790ab3e9f7cb621.pkl<br>  P88Y4517_data                    111553855  2023-01-31 06:58     cache_file_7b30f569d756c4cf.pkl<br>  P88Y4471_data                    111689851  2023-01-31 07:56     cache_file_35a2fefc584be9a.pkl<br>  P88Y5331_data                    127102647  2023-01-31 14:58     cache_file_38d2b001e1d04a4.pkl<br>  605F-0682_data                   112415205  2023-01-31 15:04     cache_file_57eb1142756274.pkl<br>  P88Y4386_data                    122106453  2023-01-31 15:10     cache_file_59da041fb71e803.pkl<br>  PSR J1357-6429_data              132529082  2023-01-31 15:23     cache_file_55b894a87820580c.pkl<br>  605P-0580_data                   120702295  2023-01-31 15:28     cache_file_c7be93c95b2c225.pkl<br>  211F-0626_data                   130126409  2023-01-31 15:34     cache_file_2633ec13ac97de82.pkl<br>  504H-0404_data                   121306969  2023-01-31 15:39     cache_file_541561e70b3b048e.pkl<br>  P88Y2490_data                     98582316  2023-01-31 15:44     cache_file_30359e29f3030d7.pkl<br>  011N-0132_data                   130847675  2023-01-31 15:49     cache_file_3278a1fa563ada9d.pkl<br>  605S-0554_data                    96249244  2023-01-31 15:55     cache_file_c434cba1b6edfef.pkl<br>  011F-0201_data                   117280982  2023-01-31 15:59     cache_file_22b9cfcb516405ae.pkl<br>  211N-0574_data                   139315015  2023-02-01 00:55     cache_file_34aa422dbf5da02c.pkl<br>  504P-0705_data                   138245107  2023-02-01 01:02     cache_file_7dfcf47234ba632d.pkl<br>  P88Y1027_data                    112130081  2023-02-01 06:53     cache_file_39da7b2847b7dd04.pkl<br>  P88Y0847_data                    131647138  2023-02-01 07:02     cache_file_3820a50309187121.pkl<br>  P88Y2347_data                    106501689  2023-02-01 07:09     cache_file_562d8a961aa8f820.pkl<br>  P88Y3998_data                     94782044  2023-02-01 07:15     cache_file_d2a4a2dcf3299.pkl<br>  P88Y4214_data                    140221389  2023-02-01 08:41     cache_file_62841df21c747e19.pkl<br>  998P-0220_data                   135136295  2023-02-01 08:59     cache_file_6384e585eae7eec4.pkl<br>  504F-1136_data                   114249982  2023-02-01 09:38     cache_file_320a1fec276c8591.pkl<br>  504P-0577_data                   126823235  2023-02-01 09:47     cache_file_74519c8a9dd6ca62.pkl<br>  605F-0288_data                   108023251  2023-02-01 09:54     cache_file_685b9feb4b485a9.pkl<br>  204P-0128_data                   131789686  2023-02-01 10:02     cache_file_3849dae136d5c69a.pkl<br>  998N-0517_data                   113347395  2023-02-04 06:30     cache_file_79bd0e3ab24b937.pkl<br>  404P-0450_data                   154289051  2023-02-04 06:47     cache_file_245009c75e3172b.pkl<br>  P88Y5536_data                    123616192  2023-02-04 06:54     cache_file_b412df00a822af6.pkl<br>  215N-0174_data                   130884915  2023-02-04 10:44     cache_file_31bcfa959c46b6b2.pkl<br>  404N-0806_data                   115835104  2023-02-04 11:17     cache_file_777b71d9750e66b3.pkl<br>  404P-0559_data                   107234295  2023-02-04 11:25     cache_file_70c217fb103fcc6b.pkl<br>  405P-0270_data                   102170670  2023-02-04 11:34     cache_file_de595efdb025d0c.pkl<br>  204N-0229_data                   113453599  2023-02-05 04:37     cache_file_5cd4d49f7a8b90a6.pkl<br>  P88Y1391_data                    112706279  2023-02-05 06:35     cache_file_55b300199a3c45b.pkl<br>  PSR J2241-5236_data               96213061  2023-02-06 10:11     cache_file_53e228f66a61d190.pkl<br>  PSR J1311-3430_data               94157455  2023-02-06 14:38     cache_file_f8c80277ad30a30.pkl<br>  PSR J2039-5617_data              102626965  2023-02-06 14:49     cache_file_fd26eff366c0d76.pkl<br>  998F-0341_data                   137830420  2023-02-06 16:52     cache_file_465c0d5be8a6189c.pkl<br>  407F-0220_data                   117710404  2023-02-06 17:00     cache_file_3e6c40f88cef2da2.pkl<br>  011N-0235_data                   113703856  2023-02-07 23:43     cache_file_3a73abe1b6f04885.pkl<br>  211N-1160_data                   113535261  2023-02-08 04:03     cache_file_6665b60c2f7fb02f.pkl<br>  011N-0234_data                   113134588  2023-02-08 04:10     cache_file_89a986c8f37c973.pkl<br>  998N-0525_data                   113156645  2023-02-08 04:18     cache_file_6998894014961f7b.pkl<br>  011F-0263_data                   113532601  2023-02-08 18:20     cache_file_67898a5756c2f61f.pkl<br>  011H-0164_data                   113221976  2023-02-08 18:26     cache_file_4389387da452dce4.pkl<br>  011N-0231_data                   113534260  2023-02-08 18:31     cache_file_e54c2a8e1ba046a.pkl<br>  03bF-0297_data                   112933065  2023-02-08 18:38     cache_file_939f7460ef7263e.pkl<br>  03bF-0300_data                   113289974  2023-02-08 18:43     cache_file_5f2bde945e4f0777.pkl<br>  03bP-0249_data                   112805308  2023-02-08 18:49     cache_file_365c84c7d62ca976.pkl<br>  211F-1054_data                   113335257  2023-02-08 18:55     cache_file_a0d6cfe8582f341.pkl<br>  211N-1188_data                   112540330  2023-02-08 19:01     cache_file_e38948e1624867d.pkl<br>  211S-0603_data                   113253371  2023-02-08 19:07     cache_file_4ba0e6b6ff92cc7.pkl<br>  211S-0618_data                   112813407  2023-02-08 19:13     cache_file_7832a817cb3e57d8.pkl<br>  404F-0756_data                   112814856  2023-02-08 19:19     cache_file_1c092abd407dd4d.pkl<br>  504P-1216_data                   112705593  2023-02-08 19:25     cache_file_a7c974815ca67b0.pkl<br>  P88Y1295_data                    113399335  2023-02-08 19:31     cache_file_37d18adbddd30b3.pkl<br>  P88Y1313_data                    113583281  2023-02-08 19:37     cache_file_874c86d9a54a6a0.pkl<br>  PSR J0540-6919_data              113568287  2023-02-08 19:44     cache_file_6b8079e04247f49.pkl<br>  S8008-4135_data                  113194445  2023-02-08 19:50     cache_file_4eea7a4b11ae629.pkl<br>  P88Y1091_data                    112387821  2023-02-08 22:22     cache_file_6e091d7a1c88860.pkl<br>  P88Y1273_data                    111263761  2023-02-08 22:28     cache_file_03c900a84ffe9cc.pkl<br>  P88Y1220_data                    111549417  2023-02-08 22:34     cache_file_75cdc74447ba01b4.pkl<br>  PSR J0218+4232_data               99995532  2023-02-09 15:31     cache_file_71fead22408c1216.pkl<br>  PSR J0102+4839_data              106937173  2023-02-09 15:43     cache_file_81669499d2ae147.pkl<br>  PSR J0251+2606_data              100707600  2023-02-09 15:54     cache_file_16513c1f29ab6bc6.pkl<br>  PSR J2215+5135_data              117768497  2023-02-09 17:23     cache_file_2a985847153417.pkl<br>  P88Y3341_data                     93942723  2023-02-10 05:46     cache_file_c079697ee403af1.pkl<br>  P88Y5818_data                    101728599  2023-02-10 05:52     cache_file_7ae18ca2a1c548d1.pkl<br>  PSR J0751+1807_data               98977431  2023-02-10 06:41     cache_file_5525f43ee0f9dd25.pkl<br>  PSR J2310-0555_data               87391059  2023-02-10 06:47     cache_file_972ca5238344d8.pkl<br>  PSR J1124-3653_data               93320402  2023-02-10 06:57     cache_file_21add1d995de3633.pkl<br>  PSR J1641+8049_data              129607898  2023-02-10 07:18     cache_file_25e66f57ef2decd.pkl<br>  PSR J1908+2105_data              117241432  2023-02-10 07:25     cache_file_e7141f39539cdb6.pkl<br>  PSR J1959+2048_data              116225109  2023-02-10 11:45     cache_file_53f70ba403f32a53.pkl<br>  PSR J0418+6635_data              132029268  2023-02-10 12:35     cache_file_22913ab7ccdde514.pkl<br>  PSR J1744-1134_data              102048499  2023-02-10 12:42     cache_file_10f1c391f2e43393.pkl<br>  PSR J0030+0451_data               90776917  2023-02-10 12:49     cache_file_484ce7cad11b40.pkl<br>  605F-0240_data                   100598267  2023-02-11 12:32     cache_file_67873421a5713ba6.pkl<br>  PSR J2129-0429_data               90967611  2023-02-11 13:19     cache_file_56e5734b6adb3ab1.pkl<br>  P88Y0114_data                    107246118  2023-02-11 16:33     cache_file_44ce36eba3af4fe5.pkl<br>  P88Y0266_data                    108077179  2023-02-11 16:39     cache_file_78150e49d4737dd5.pkl<br>  03bN-0296_data                    97656888  2023-02-11 16:44     cache_file_3efa434ae0ad7702.pkl<br>  P88Y4343_data                     95531436  2023-02-11 16:50     cache_file_1554a7e40a69ddb.pkl<br>  P88Y4390_data                     95939018  2023-02-11 16:56     cache_file_ddc65594cb5658e.pkl<br>  P88Y4422_data                    102046000  2023-02-11 17:01     cache_file_2ea5f95cfeb4a86.pkl<br>  P88Y4656_data                    133509859  2023-02-11 17:07     cache_file_517598b7c20c02e.pkl<br>  P88Y4668_data                     97419791  2023-02-11 17:12     cache_file_19dc784632c59c1a.pkl<br>  605F-0287_data                   106520960  2023-02-11 17:19     cache_file_b42907e3f9ba81b.pkl<br>  P88Y5295_data                    105824754  2023-02-11 17:25     cache_file_473949346935469a.pkl<br>  P88Y3389_data                     98098126  2023-02-12 05:28     cache_file_6665769b0dde821.pkl<br>  P88Y3906_data                     95075337  2023-02-12 05:34     cache_file_3f349a869cd81bc1.pkl<br>  504S-1503_data                   111795481  2023-02-12 05:40     cache_file_c2f7a7edfa119af.pkl<br>  P88Y4696_data                    102489709  2023-02-12 05:48     cache_file_d9af707459eda07.pkl<br>  P88Y4783_data                    106635494  2023-02-12 05:54     cache_file_6a14945c7f705a8.pkl<br>  P88Y4876_data                    126572621  2023-02-12 06:01     cache_file_66c9a67796b2cf00.pkl<br>  PSR J1823-3021A_data              99218091  2023-02-12 06:08     cache_file_35d3ab1d76bb3e4f.pkl<br>  PSR J1835-3259B_data              96173959  2023-02-12 10:32     cache_file_34f3e12c5c6da69.pkl<br>  P88Y5171_data                    128298842  2023-02-12 10:37     cache_file_9fdd9fc353c53f.pkl<br>  211F-0710_data                   109077276  2023-02-12 10:42     cache_file_7aeeddb3d128a85a.pkl<br>  504P-0895_data                   116030488  2023-02-12 10:48     cache_file_757e63544b4455f.pkl<br>  504N-1372_data                    99763650  2023-02-12 10:54     cache_file_30fec19a020f04aa.pkl<br>  605H-0192_data                   112636328  2023-02-12 10:59     cache_file_f3b113d280e3d47.pkl<br>  P88Y5973_data                     91076839  2023-02-12 11:04     cache_file_4ba9d7de69207b8.pkl<br>  011N-0259_data                    92492204  2023-02-12 11:09     cache_file_c052181452c677b.pkl<br>  03bP-0194_data                    99137339  2023-02-15 09:16     cache_file_22a8b838b54e540.pkl<br>  03bN-0316_data                   105644903  2023-02-16 04:53     cache_file_70944577b0b6c0fd.pkl<br></pre></div> </details>


In [9]:
len(Config().cache)

435

## Time conversion

- MET: mission elapsed time
- MJD: modified Julian date (days)
- UTC: ISO time
- UTCnow: current ISO time
- BarycentricTime

In [10]:
#| export

day = 24*3600.
first_data=54683
#mission_start = Time('2001-01-01T00:00:00', scale='utc').mjd
# From a FT2 file header
# MJDREFI =               51910. / Integer part of MJD corresponding to SC clock S
# MJDREFF =  0.00074287037037037 / Fractional part of MJD corresponding to SC cloc
mission_start = 51910.00074287037
from datetime import datetime
from astropy.time import Time

def MJD(arg):
    """ convert MET or UTC to MJD
    """

    if type(arg)==str:
        if arg=='now':
            return Time(datetime.utcnow()).mjd
        while len(arg.split('-'))<3:
            arg+='-1'
        return Time(arg, format='iso').mjd
    return (mission_start + arg/day  )

def UTC(mjd):
    " convert MJD value to ISO date string"
    t=Time(mjd, format='mjd')
    t.format='iso'; t.out_subfmt='date_hm'
    return t.value

def UTCnow():
    """ current UTC """
    t=datetime.utcnow()
    return f'UTC {t.year}-{t.month:02d}-{t.day} {t.hour:02d}:{t.minute:02d}'

def mission_week(mjd):
    """ return the mission week number for a MJD value
    (Note that week #0 starts on UTC Thursday 2008-05-29 00:00, the first data is in week 9, and that week 525 is missing)
    """
    return (mjd-54615)//(7)

def mjd_range(start,stop, make_round=True):
    """Return a tuple with a valid MJD range
    If either is greater than first_data, interpret as MJD
    otherwise offset from first_data, or now

    So 0,0 is full range, -7, 0 is last 7 days, 0,7 is first 7 days

    """
    a,b = start, stop
    # extract MJD range
    now = MJD('now')
    if a<0: a = now-a
    elif a < first_data : a+=first_data
    if b <= 0 :  b = now-b
    elif b < first_data: b += first_data

    return (round(a),round(b)) if make_round else (a,b)

In [11]:
UTC(MJD(0)), UTC(first_data), UTCnow(), MJD('now'), UTC(MJD('now')), mjd_range(0,0)

('2001-01-01 00:01',
 '2008-08-05 00:00',
 'UTC 2023-02-17 21:54',
 59992.91319206486,
 '2023-02-17 21:54',
 (54683, 59993))

In [12]:
np.arange(*[mission_week(x) for x in mjd_range(0,100)])

array([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])

In [13]:
assert UTC(MJD(0))=='2001-01-01 00:01'
assert MJD('2008')==54466

In [14]:
#| export
class FermiInterval():
    """For iteration thru (start,stop) tuples in the Fermi data set
    """
    def __init__(self, interval=30, offset=0):
        from wtlike.config import  MJD, first_data
        self.interval=interval
        a,b = first_data+offset, MJD('now')
        self.mm = np.arange(a,round(b),interval)

    def __len__(self):
        return len(self.mm)-1

    def __getitem__(self, k):
        """Return a (start, end) MJD tuple for the kth interval, k=0...N-1. 
        if k<0, return the range for the full interval.
        """
        return (self.mm[k], self.mm[k+1],) if k>=0 else (self.mm[k-1],self.mm[k])

    def __call__(self, k):
        """ Return a timeinterval tuple
        """
        a,b = self[k]
        return (a,b,self.interval)

class FermiMonth(FermiInterval):
    def __init__(self):
        super().__init__(interval=30)

class FermiWeek(FermiInterval):
    def __init__(self):
        super().__init__(interval=7)

In [15]:
#| export
class BaryCentricTime:
    def __init__(self, skypos):
        """ Manage calculation of light travel time
        
        See [astropy.time](https://docs.astropy.org/en/stable/time/index.html#barycentric-and-heliocentric-light-travel-time-corrections)
        We use the center of the Earth for the location here.

        """
        self.skypos = skypos
        self.elat = skypos.barycentricmeanecliptic.lat.deg # for reference

    def __call__(self, times):
        """return light travel time (d) for times in MJD
        """
        from astropy import time, coordinates as coord
        etimes = time.Time(times, format='mjd',  scale='utc', 
            location=coord.EarthLocation(0,0,0))
        return etimes.light_travel_time(self.skypos).jd

## Miscellaneous utilities
- bins_size_name
- decorate_with
- Timer

In [16]:
#| export
def bin_size_name(bins):
    """Provide a nice name, e.g., 'day' for a time interval
    """
    if np.isscalar(bins) :
        binsize = bins
    else:
        binsize = np.mean(bins)

    def check_unit(x):
        unit_table = dict(year=1/365.25, week=1/7, day=1, hour=24, min=24*60) #, min=24*60, s=24*3600)
        for name, unit in unit_table.items():
            t = x*unit
            r = np.mod(t+1e-9,1)

            if r<1e-6 : #or t>1:
                return t, name
        return x, 'day'
    n, unit =  check_unit(binsize)
    nt = f'{n:.0f}' if np.mod(n,1)<1e-3 else f'{n:.1f}'
    return f'{nt}-{unit}'# if n>1 else f'{unit}'

In [17]:
print([bin_size_name(x) for x in (1/96, 1,2,7, 14, 365.25)])

['15-min', '1-day', '2-day', '1-week', '2-week', '1-year']


In [18]:
#| export
def decorate_with(other_func):
    def decorator(func):
        func.__doc__ += other_func.__doc__
        return func
    return decorator

In [19]:
#| export
import time

class Timer():
    """Usage:
    ```
    with Timer() as t:
        time.sleep(5)
    print(t)
    ```
    """

    def __init__(self):
        self.t=time.time()
        self.exit_time=1e6

    def __enter__(self):
        return self
    def __exit__(self, *pars):
        self.exit_time = time.time()-self.t
    def __repr__(self):
         return  f'elapsed time: {self.elapsed:.1f}s ({self.elapsed/60:.1f} min)'
    @property
    def elapsed(self):
        return min(time.time()-self.t, self.exit_time)

In [20]:
#| code-fold: show
with Timer() as t:

    time.sleep(2)
    print('intemediate',t)
    time.sleep(3)
print('Final',t)
assert(abs(t.elapsed -5)<0.1), f'wrong elapsed time: {t.elapsed}'

intemediate elapsed time: 2.0s (0.0 min)
Final elapsed time: 5.0s (0.1 min)


In [21]:
#| include: false
from nbdev import nbdev_export
nbdev_export()
!date

Fri Feb 17 13:55:07 PST 2023
